# COVID-19 Impacts Analysis

Following [this article](https://thecleverprogrammer.com/2022/04/19/covid-19-impacts-analysis-using-python/) from Aman Kharwal.

Dataset can be found [at Kaggle](https://www.kaggle.com/datasets/shashwatwork/impact-of-covid19-pandemic-on-the-global-economy).

In [97]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

data = pd.read_csv("data/transformed_data.csv")
data.head()

,CODE,COUNTRY,DATE,HDI,TC,TD,STI,POP,GDPCAP
0,AFG,Afghanistan,2019-12-31,0.498,0.0,0.0,0.0,17.477233,7.497754
1,AFG,Afghanistan,2020-01-01,0.498,0.0,0.0,0.0,17.477233,7.497754
2,AFG,Afghanistan,2020-01-02,0.498,0.0,0.0,0.0,17.477233,7.497754
3,AFG,Afghanistan,2020-01-03,0.498,0.0,0.0,0.0,17.477233,7.497754
4,AFG,Afghanistan,2020-01-04,0.498,0.0,0.0,0.0,17.477233,7.497754


In [98]:
raw_data = pd.read_csv('data/raw_data.csv')
raw_data.head()

,iso_code,location,date,total_cases,total_deaths,stringency_index,population,gdp_per_capita,human_development_index,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,AFG,Afghanistan,2019-12-31,0.0,0.0,0.0,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494
1,AFG,Afghanistan,2020-01-01,0.0,0.0,0.0,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494
2,AFG,Afghanistan,2020-01-02,0.0,0.0,0.0,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494
3,AFG,Afghanistan,2020-01-03,0.0,0.0,0.0,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494
4,AFG,Afghanistan,2020-01-04,0.0,0.0,0.0,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494


## Aggregating the data (transformed_data + raw_data)

In [99]:
# Getting the mode value
mode_value = data['COUNTRY'].value_counts().mode()
mode_value = mode_value.get(0)
mode_value

294

- The mode is the value that appears most often in a set of data values.

In [100]:
code = data['CODE'].unique().tolist()
countries = data['COUNTRY'].unique().tolist()
hdi = []
total_cases = []
total_deaths = []
sti = []
population = data['POP'].unique().tolist()
gdp = []

for country in countries:
    hdi.append((data.loc[data['COUNTRY'] == country, 'HDI']).sum() / mode_value)
    total_cases.append((raw_data.loc[raw_data['location'] == country, 'total_cases']).sum())
    total_deaths.append((raw_data.loc[raw_data['location'] == country, 'total_deaths']).sum())
    sti.append((data.loc[data['COUNTRY'] == country, 'STI']).sum() / mode_value)
    population.append((raw_data.loc[raw_data['location'] == country, 'population']).sum() / mode_value)
    
aggregated_data = list(zip(code, countries, hdi, total_cases, total_deaths, sti, population))

In [101]:
# Creating pd.DataFrame
aggregated_data = pd.DataFrame(
    aggregated_data, 
    columns = [
        "Country Code", 
        "Country", 
        "HDI", 
        "Total Cases", 
        "Total Deaths", 
        "Stringency Index", 
        "Population"])

aggregated_data.head()

,Country Code,Country,HDI,Total Cases,Total Deaths,Stringency Index,Population
0,AFG,Afghanistan,0.498000,5126433.0,165875.0,3.049673,17.477233
1,ALB,Albania,0.600765,1071951.0,31056.0,3.005624,14.872537
2,DZA,Algeria,0.754000,4893999.0,206429.0,3.195168,17.596309
3,AND,Andorra,0.659551,223576.0,9850.0,2.677654,11.254996
4,AGO,Angola,0.418952,304005.0,11820.0,2.965560,17.307957


## Top 10 Countries with Highest Covid Cases

In [102]:
top10_countries_by_cases = aggregated_data.sort_values(by=['Total Cases'], ascending=False).head(10)
top10_countries_by_cases

,Country Code,Country,HDI,Total Cases,Total Deaths,Stringency Index,Population
200,USA,United States,0.924000,746014098.0,26477574.0,3.350949,19.617637
27,BRA,Brazil,0.759000,425704517.0,14340567.0,3.136028,19.174732
90,IND,India,0.640000,407771615.0,7247327.0,3.610552,21.045353
157,RUS,Russia,0.816000,132888951.0,2131571.0,3.380088,18.798668
150,PER,Peru,0.599490,74882695.0,3020038.0,3.430126,17.311165
125,MEX,Mexico,0.774000,74347548.0,7295850.0,3.019289,18.674802
178,ESP,Spain,0.887969,73717676.0,5510624.0,3.393922,17.660427
175,ZAF,South Africa,0.608653,63027659.0,1357682.0,3.364333,17.898266
42,COL,Colombia,0.581847,60543682.0,1936134.0,3.357923,17.745037
199,GBR,United Kingdom,0.922000,59475032.0,7249573.0,3.353883,18.033340


In [103]:
top10_countries_by_cases['GDP Before Covid'] = [
    65279.53, 8897.49, 2100.75, 11497.65, 7027.61, 9946.03, 29564.74, 6001.40, 6424.98, 42354.41]
top10_countries_by_cases['GDP During Covid'] = [
    63543.58, 6796.84, 1900.71, 10126.72, 6126.87, 8346.70, 27057.16, 5090.72, 5332.77, 40284.84]
top10_countries_by_cases

,Country Code,Country,HDI,Total Cases,Total Deaths,Stringency Index,Population,GDP Before Covid,GDP During Covid
200,USA,United States,0.924000,746014098.0,26477574.0,3.350949,19.617637,65279.53,63543.58
27,BRA,Brazil,0.759000,425704517.0,14340567.0,3.136028,19.174732,8897.49,6796.84
90,IND,India,0.640000,407771615.0,7247327.0,3.610552,21.045353,2100.75,1900.71
157,RUS,Russia,0.816000,132888951.0,2131571.0,3.380088,18.798668,11497.65,10126.72
150,PER,Peru,0.599490,74882695.0,3020038.0,3.430126,17.311165,7027.61,6126.87
125,MEX,Mexico,0.774000,74347548.0,7295850.0,3.019289,18.674802,9946.03,8346.70
178,ESP,Spain,0.887969,73717676.0,5510624.0,3.393922,17.660427,29564.74,27057.16
175,ZAF,South Africa,0.608653,63027659.0,1357682.0,3.364333,17.898266,6001.40,5090.72
42,COL,Colombia,0.581847,60543682.0,1936134.0,3.357923,17.745037,6424.98,5332.77
199,GBR,United Kingdom,0.922000,59475032.0,7249573.0,3.353883,18.033340,42354.41,40284.84


## Analyzing the Spread of Covid-19

In [104]:
figure = px.bar(top10_countries_by_cases, y='Total Cases', x='Country', title='Countries with Highest Covid Cases')
figure.show()

In [105]:
figure = px.bar(top10_countries_by_cases, y='Total Deaths', x='Country', title='Countries with Highest Deaths')
figure.show()

In [106]:
figure = go.Figure()
figure.add_trace(go.Bar(
    x=top10_countries_by_cases['Country'],
    y=top10_countries_by_cases['Total Cases'],
    name='Total Cases',
    marker_color='red'))

figure.add_trace(go.Bar(
    x=top10_countries_by_cases['Country'],
    y=top10_countries_by_cases['Total Deaths'],
    name='Total Deaths',
    marker_color='blue'))

figure.update_layout(barmode='group', xaxis_tickangle=-45)
figure.show()

In [107]:
cases = top10_countries_by_cases['Total Cases'].sum()
deaths = top10_countries_by_cases['Total Deaths'].sum()
labels = ['Total Cases', 'Total Deaths']
values = [cases, deaths]

figure = px.pie(top10_countries_by_cases, values=values, names=labels, title='Percentage of Total Cases and Deaths')
figure.show()

In [108]:
death_rate = (top10_countries_by_cases["Total Deaths"].sum() / top10_countries_by_cases["Total Cases"].sum()) * 100
print(f"Death rate: {death_rate}")

Death rate: 3.6144212045653767


In [109]:
figure = px.bar(
    top10_countries_by_cases,
    x='Country', 
    y='Total Cases', 
    hover_data=['Population', 'Total Deaths'], 
    color='Stringency Index', 
    height=400, 
    title='Stringency Index during COVID-19')

figure.show()

Here we can see that India is performing well in the stringency index during the outbreak of covid-19.

## Analyzing Covid-19 Impacts on Economy

In [110]:
figure = px.bar(
    top10_countries_by_cases,
    x='Country',
    y='Total Cases',
    hover_data=['Population', 'Total Deaths'], 
    color='GDP Before Covid', 
    height=400, 
    title="GDP Per Capita Before Covid-19")

figure.show()

In [111]:
figure = px.bar(
    top10_countries_by_cases,
    x='Country',
    y='Total Cases',
    hover_data=['Population', 'Total Deaths'], 
    color='GDP During Covid', 
    height=400, 
    title="GDP Per Capita During Covid-19")

figure.show()

In [112]:
figure = go.Figure()

figure.add_trace(go.Bar(
    x=top10_countries_by_cases["Country"],
    y=top10_countries_by_cases["GDP Before Covid"],
    name='GDP Per Capita Before Covid-19',
    marker_color='red'
))

figure.add_trace(go.Bar(
    x=top10_countries_by_cases["Country"],
    y=top10_countries_by_cases["GDP During Covid"],
    name='GDP Per Capita During Covid-19',
    marker_color='blue'
))

figure.update_layout(barmode='group', xaxis_tickangle=-45)
figure.show()

## Analysing COVID-19 impacts on Human Development Indexes

In [113]:
figure = px.bar(
    top10_countries_by_cases, 
    x='Country', 
    y='Total Cases',
    hover_data=['Population', 'Total Deaths'], 
    color='HDI', 
    height=400, 
    title="Human Development Index during Covid-19")

figure.show()